# XX网新闻推荐服务

## 项目背景与实现
参考音乐推荐即可。

## 数据集说明


## 导入需要的库

In [1]:
import numpy as np
import pandas as pd
import re
import jieba
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter

## 原始数据的处理
数据集是一个xml类型的文件，我们不能使用pandas直接进行加载。  
我们读取原始的数据集，使用正则表达式提取`<contenttitle>`与`</contenttitle>`中的标题信息。并将信息输入到另外一个文档中。

In [2]:
re_obj = re.compile(r"<contenttitle>(.+?)</contenttitle>")
re_obj2 = re.compile(r"<content>(.+?)</content>")
with open("news_tensite_xml.dat", encoding="ANSI") as f, open("news.dat", "wt", encoding="ANSI") as f2:
    for line in f:
        match = re_obj.match(line)
        if match:
            f2.write(match.group(1))
            line2 = f.readline()
            match2 = re_obj2.match(line2)
            if match2:
                f2.write(match2.group(1))
            f2.write("\n")

## 加载数据集
加载数据集，并查看数据的基本信息。

In [3]:
news = pd.read_csv(r"news.dat", header=None, names=["title"], encoding="ANSI")

In [7]:
display(news.shape)
display(news.head())
# news.iloc[0].tolist()

(1293605, 1)

,title
0,公安机关销毁１０余万非法枪支 跨国武器走私渐起中广网唐山６月１２日消息（记者汤一亮 庄胜春）...
1,张绍刚发道歉信网友不认可：他的问题是俯视他人（图）天津卫视求职节目《非你莫属》“晕倒门”事件...
2,＃（关注夏收）（３）夫妻“麦客”忙麦收临沂（山东），２０１２年６月４日 夫妻“麦客”忙麦收 ...
3,欧洲杯大战在即 荷兰葡萄牙面临淘汰将背水一战中广网北京６月１３日消息（记者王宇）据中国之声《...
4,扎克伯格携妻罗马当街吃３０元麦当劳午餐（组图）环球网记者李亮报道，正在意大利度蜜月的“脸谱”...


## 数据清洗

### 缺失值处理

In [8]:
news.isnull().sum()

title    0
dtype: int64

### 重复值处理

In [10]:
# news.duplicated().sum()
# news[news.duplicated()]
news.drop_duplicates(inplace=True)
display(news.shape)

(995364, 1)

### 去标点与分词处理

In [11]:
re_obj = re.compile(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~——！，。？、￥…（）：【】《》‘’“”\s]+")

def process_text(text):
    text = re_obj.sub("", text)
    return jieba.lcut(text)

In [12]:
# 因为执行时间较长，这里，我们只取前1000条新闻。
news = news.iloc[:1000]
news["title"] = news["title"].apply(process_text)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.476 seconds.
Prefix dict has been built succesfully.


In [13]:
news.head()

,title
0,"[公安机关, 销毁, １, ０, 余万, 非法, 枪支, 跨国, 武器, 走私, 渐起, 中..."
1,"[张绍, 刚发, 道歉信, 网友, 不, 认可, 他, 的, 问题, 是, 俯视, 他人, ..."
2,"[＃, 关注, 夏收, ３, 夫妻, 麦客, 忙, 麦收, 临沂, 山东, ２, ０, １,..."
3,"[欧洲杯, 大战, 在, 即, 荷兰, 葡萄牙, 面临, 淘汰, 将, 背水一战, 中广网,..."
4,"[扎克, 伯格, 携妻, 罗马, 当街, 吃, ３, ０, 元, 麦当劳, 午餐, 组图, ..."


In [15]:
ord("１"), ord("1")

(65297, 49)

## 数据建模

In [ ]:
documents = news["title"].tolist()
documents[:5]

In [17]:
documents = news["title"].tolist()
# doc2vec训练的语料与word2vec的格式有些不同。doc2vec要求语料为TaggedDocument构成的数组。
# TaggedDocument代表的就是一篇文档，该文档包含若干个词汇。

# TaggedDocument的参数。 doc 一维数组，文档中的词汇。 [index] 表示文档的标签。
documents = [TaggedDocument(doc, [index]) for index, doc in enumerate(documents)]
# documents：用来训练的语料。要求是TaggedDocument构成的数组。
# vector_size：文档进行嵌入的维度。（文档映射到空间当中，向量的维度）。
# window：当前词汇与预测词汇之间的最远距离。
# min_count: 考虑最小词频的词汇。（如果词汇低于该词频，则不再考虑该词汇）
# worker：并发的数量。
# epochs：训练的轮数。
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=50)

In [25]:
# documents[0].words
# documents[0].tags
vector = model.infer_vector(documents[0].words)
sim = model.docvecs.most_similar([vector], topn=5)
sim

[(0, 0.9714878797531128),
 (78, 0.846230685710907),
 (831, 0.6213095784187317),
 (319, 0.6159457564353943),
 (922, 0.6107473373413086)]

## 模型效果评估

In [29]:
ranks = []
for doc in documents:
    # doc.words就是一个文档中包含的词汇列表。infer_vector方法的作用是将参数指定的
    # 文档中的词汇列表表示成向量的形式（推断表示）。即如果参数指定的词汇(文档)映射到
    # 当前的模型空间中，应该表示成为怎样的向量。
    inferred_vector = model.infer_vector(doc.words)
    # 根据参数指定的向量，返回当前模型中，与参数向量最相似的向量。
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    # 查看推断文档的向量与自己比较，相似度排行第几。（理想情况下，应该排在最前面）
    rank = [tag for tag, _ in sims].index(doc.tags[0])
    ranks.append(rank)

我们可以输出查看下模型的效果。

In [26]:
Counter([1, 2, 3, 1, 2])

Counter({1: 2, 2: 2, 3: 1})

In [30]:
print(Counter(ranks))

Counter({0: 975, 1: 10, 2: 3, 3: 2, 7: 2, 19: 1, 28: 1, 13: 1, 24: 1, 259: 1, 132: 1, 31: 1, 35: 1})


## 模型预测

In [34]:
display(",".join(documents[0].words))
v = model.infer_vector(documents[0].words)
# display(v)
similar = model.docvecs.most_similar([v], topn=3)
for index, score in similar:
    display(score, ",".join(documents[index].words))

'公安机关,销毁,１,０,余万,非法,枪支,跨国,武器,走私,渐起,中广网,唐山,６,月,１,２,日,消息,记者,汤一亮,庄胜春据,中国,之声,新闻,晚,高峰,报道,今天,１,２,日,上午,公安机关,２,０,１,２,年缉枪,制爆,专项,行动,统一,销毁,非法,枪爆,物品,活动,在,河北,唐山,正式,启动,１,０,万余只,非法,枪支,２,５,０,余吨,炸药,在,全国,１,５,０,个,城市,被,统一,销毁,\ue40c,黄明,现在,我,宣布,全国,缉枪,制爆,统一,销毁,行动,开始,\ue40c,随着,公安部,副,部长,黄明,一声令下,大量,仿制,式,枪,以及,猎枪,火药枪,气枪,在,河北,唐山,钢铁厂,被,投入,炼钢炉,与此同时,在,全国,各,省区市,１,５,０,个,城市,破案,追缴,和,群众,主动,上缴,的,１,０,万余支,非法,枪支,被,集中,销毁,在,全国,各,指定,场所,２,５,０,余吨,炸药,被,分别,销毁,公安部,治安,局局长,刘,绍武,介绍,这次,销毁,的,非法,枪支,来源于,三个,方面,\ue40c,刘,绍武,打击,破案,包括,涉黑,涉恶,的,团伙,犯罪,毒品,犯罪,还有,从,境外,非法,走私,的,枪支,爆炸物,\ue40c,在,销毁,现场,记者,看到,了,被,追缴,和,上缴,的,各式各样,的,枪支,\ue40c,刘,绍武,也,包括,制式,枪,有的是,军用,枪,仿制,的,制式,抢,还有,猎枪,私制,的,火药枪,等等,按照,我国,的,枪支,管理法,这些,都,是,严厉,禁止,个人,非法,持有,的,中国,是,世界,上,持枪,犯罪,的,犯罪率,最低,的,国家,之一,\ue40c,中,美,联手,破获,特大,跨国,走私,武器弹药,案,\ue40c,近日,中,美,执法,部门,联手,成功,破获,特大,跨国,走私,武器弹药,案,在,中国,抓获,犯罪,嫌疑人,２,３,名,缴获,各类,枪支,９,３,支,子弹,５,万余,发及,大量,枪支,配件,在,美国,抓获,犯罪,嫌疑人,３,名,缴获,各类,枪支,１,２,支,这,是,公安部,与,美国移民,海关,执法局,通过,联合,调查,方式,侦破,重大,跨国,案件,的,又,一,成功,案例,\ue40c,２,０,１,１,年,８,月,２,５,日,上海浦东国际机场,海关,在,对,美国纽约,发,往,浙江,台州,申报,品名,为,扩音器,音箱,的,快件

0.9725781679153442

'公安机关,销毁,１,０,余万,非法,枪支,跨国,武器,走私,渐起,中广网,唐山,６,月,１,２,日,消息,记者,汤一亮,庄胜春据,中国,之声,新闻,晚,高峰,报道,今天,１,２,日,上午,公安机关,２,０,１,２,年缉枪,制爆,专项,行动,统一,销毁,非法,枪爆,物品,活动,在,河北,唐山,正式,启动,１,０,万余只,非法,枪支,２,５,０,余吨,炸药,在,全国,１,５,０,个,城市,被,统一,销毁,\ue40c,黄明,现在,我,宣布,全国,缉枪,制爆,统一,销毁,行动,开始,\ue40c,随着,公安部,副,部长,黄明,一声令下,大量,仿制,式,枪,以及,猎枪,火药枪,气枪,在,河北,唐山,钢铁厂,被,投入,炼钢炉,与此同时,在,全国,各,省区市,１,５,０,个,城市,破案,追缴,和,群众,主动,上缴,的,１,０,万余支,非法,枪支,被,集中,销毁,在,全国,各,指定,场所,２,５,０,余吨,炸药,被,分别,销毁,公安部,治安,局局长,刘,绍武,介绍,这次,销毁,的,非法,枪支,来源于,三个,方面,\ue40c,刘,绍武,打击,破案,包括,涉黑,涉恶,的,团伙,犯罪,毒品,犯罪,还有,从,境外,非法,走私,的,枪支,爆炸物,\ue40c,在,销毁,现场,记者,看到,了,被,追缴,和,上缴,的,各式各样,的,枪支,\ue40c,刘,绍武,也,包括,制式,枪,有的是,军用,枪,仿制,的,制式,抢,还有,猎枪,私制,的,火药枪,等等,按照,我国,的,枪支,管理法,这些,都,是,严厉,禁止,个人,非法,持有,的,中国,是,世界,上,持枪,犯罪,的,犯罪率,最低,的,国家,之一,\ue40c,中,美,联手,破获,特大,跨国,走私,武器弹药,案,\ue40c,近日,中,美,执法,部门,联手,成功,破获,特大,跨国,走私,武器弹药,案,在,中国,抓获,犯罪,嫌疑人,２,３,名,缴获,各类,枪支,９,３,支,子弹,５,万余,发及,大量,枪支,配件,在,美国,抓获,犯罪,嫌疑人,３,名,缴获,各类,枪支,１,２,支,这,是,公安部,与,美国移民,海关,执法局,通过,联合,调查,方式,侦破,重大,跨国,案件,的,又,一,成功,案例,\ue40c,２,０,１,１,年,８,月,２,５,日,上海浦东国际机场,海关,在,对,美国纽约,发,往,浙江,台州,申报,品名,为,扩音器,音箱,的,快件

0.8513802886009216

'中,美,联手,破获,跨国,走私,武器,案,主犯,系,美国士兵,新,京报,讯,记者,邢世伟,公安部,昨日,消息,近日,中,美,执法,部门,联手,成功,破获,特大,跨国,走私,武器弹药,案,在,中国,抓获,犯罪,嫌疑人,２,３,名,缴获,各类,枪支,９,３,支,其中,制式,枪支,６,６,支,子弹,５,万余,发及,大量,枪支,配件,在,美国,抓获,犯罪,嫌疑人,３,名,缴获,各类,枪支,１,２,支,\ue40c,申报,品名,为,扩音器,实为,枪支,\ue40c,公安部,方面,透露,２,０,１,１,年,８,月,２,５,日,上海浦东国际机场,海关,在,对,美国纽约,发,往,浙江,台州,申报,品名,为,扩音器,音箱,的,快件,进行,查验,时,发现,藏有,手枪,９,支,枪支,配件,９,件,长枪,部件,７,件,经,检验,上述,物品,均,为,具有,杀伤力,的,制式,枪支,及其,配件,\ue40c,公安部,方面,称,专案组,于,８,月,２,６,日,在,浙江,台州,Ｕ,Ｐ,Ｓ,取件,处,将,犯罪,嫌疑人,王挺,男,３,２,岁,台州市,人,抓获,随后,以王,挺,为,突破口,对,其,买卖,枪支,的,上线,下线,进行,全面,梳理,核查,\ue40c,枪支,从,纽约,快递,国内,中间人,\ue40c,经查,王挺,在,一,境外,网站,上,搭识,上家,林志富,男,２,５,岁,福建省,福州市,闽侯县,人,２,０,０,９,年,１,１,月,以来,长期,居住,美国,后,进而,成为,其,走私,境外,武器弹药,入境,的,中间人,\ue40c,林志富,通过,互联网,组建,了,一个,走私,贩卖,私藏枪支,弹药,的,群体,借此,在,国内,寻找,枪支弹药,买家,并,通过,美国,Ｕ,Ｐ,Ｓ,联邦,速递,公司,将,枪支弹药,从,纽约,快,递给,多名,类似,王挺,的,中间人,再,通过,中间人,发送给,国内,买家,\ue40c,两名,中国,人,在,美,充当,中间人,\ue40c,针对,涉案,枪支,主要,来源于,美国,的,情况,公安部,及时,通报,美方,商请,美方,彻查,枪源,美方,执法,部门,于,今年,１,月,派出,移民,海关,执法局,国土,安全,调查组,赴,上海,与,我国,警方,会晤,共同,商议,联合,执法,事宜,随后,美方,顺线,查获,了,在,美国,非法,从事,枪支,贩卖,活动,的,林志富,和,李力,连,男,２,３,岁,

0.6251641511917114

'意大利,警方,查获,黄金,诈骗,逃税,大案,新华网,米兰,６,月,１,３,日电,记者,宋建,意大利,警方,近日,破获,一起,历时,数年,的,黄金,诈骗,逃税,大案,涉及,虚报,出口,销售,黄金,总量,达,３,．,３,吨,逃税,金额,逾,７,０,０,０,万,欧元,约合,５,．,６,亿元,人民币,\ue40c,据,安莎社,报道,犯罪分子,涉嫌,制造,黄金,出口,假象,以,逃避,境内,黄金,交易,税款,共有,１,５,名,嫌疑人,受到,起诉,被,指控,涉嫌,走私,非法占有,和,偷逃税,等,其中,７,人,是,从事,海关,业务,代理,的,工作人员,\ue40c,警方,称,此案,主要,涉及,意大利,北部,重要,的,金饰,工业区,维琴察,的,６,家,黄金,首饰店,在,侦破,此案,过程,中,意大利,警方,曾,赴,俄罗斯,克罗地亚,等,地,进行,调查,查出,过去,两年,内,维琴察,黄金饰品,行业,逾,３,．,７,亿,欧元,约合,２,９,．,６,２,亿元,人民币,的,违法,交易,记录,\ue40c,维琴察,市,位于,意大利,威尼托,大区,距,威尼斯,以西,６,１,公里,集中,了,意大利,最,重要,的,金饰,工业区,维琴察,黄金,珠宝展,与,瑞士,巴塞尔,珠宝展,和,美国,拉斯韦加斯,珠宝展,并,称为,世界,三大,珠宝展,是,全球,最早,发布,来,年,黄金珠宝,流行,趋势,的,重要,展会,\ue40c,去年,上任,的,意大利,总理,蒙蒂,５,月份,视察,税务,机构,时,表示,打击,偷漏税,是,意大利政府,的,中心,目标,这,对,意大利,的,国际,信誉,具有,重要,意义,\ue40c,作者,宋建'